In [1]:
# BLOK 1 
# IMPORTOVANIE KNIZNIC

import pandas as pd
import numpy as np
import json
from random import randint

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Convolution1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, GRU, Concatenate
import tensorflow.keras.utils 
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from ast import literal_eval
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report

from tensorflow.keras.models import Sequential

C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [ ]:
# ----------------------- SYNTHETIC LIGHT CURVES ----------------------- #
# ----------------------- SYNTHETIC LIGHT CURVES ----------------------- #
# ----------------------- SYNTHETIC LIGHT CURVES ----------------------- #

In [ ]:
# BLOK 2 
# Nacitanie datasetu syntetickych kriviek

data_syn = pd.read_csv(r"C:\Users\nemci\OneDrive\Počítač\Bakalárska práca\Data\df_all.csv_v2")

In [ ]:
# BLOK 3
# Rozdelenie syntetickych kriviek na podmnozinu neskrvnitych a skvrnitych

data_NoSpotty = data_syn[data_syn["spotty"] == 0]
data_Spotty = data_syn[data_syn["spotty"] == 1]

print("No spotty light curves: ", data_NoSpotty.shape[0]) 
print("Spotty light curves: ", data_Spotty.shape[0])

In [ ]:
# BLOK 4
# Diskretizacia 

newData = []
for row in data_NoSpotty['generic_bessell_v']:
    row = json.loads(row)
    minValueOfRow, maxValueOfRow = min(row), max(row)
    newRow = []
    for valueIndex in range(len(row)):
        row[valueIndex] = (row[valueIndex] - minValueOfRow) / (maxValueOfRow - minValueOfRow)
        newRow.append([row[valueIndex]])
    newData.append(newRow)
newData = np.array(newData)

In [ ]:
# BLOK 5
# Vytvorenie cieloveho atributu kriviek. Hodnota 0 pre dotykove, hodnota 1 pre oddelene krivky

target = []
oc, dt = 0, 0
for row in data_NoSpotty["morphology"]:
    if row == 'over-contact':
        target.append(0)
        oc += 1
    elif row == 'detached':
        target.append(1)
        dt += 1
target = np.array(target)
print('Over-contact no spotty: ' + str(oc) + '\n' + 'Detached no spotty: ' + str(dt))

In [ ]:
# BLOK 6
# Rozdelenie upravenych kriviek na trenovaciu a testovaciu mnozinu v pomere 80:20

print("Number of light curves: " + str(len(newData)) + "\nTarget attribute: " + str(len(target)))
X_train, X_test, y_train, y_test = train_test_split(newData, target, test_size=0.2)
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)
print("Length X_train: ", str(len(X_train)))
print("Length y_train: ", str(len(y_train)))
print("Length X_test: ", str(len(X_test)))
print("Length y_test: ", str(len(y_test)))

In [ ]:
# BLOK 7
# Vytvorenie architektúry klasifikačneho modelu a jeho trenovanie

model = Sequential()
model.add(Convolution1D(64, 3, activation = 'relu', input_shape = (100, 1))) 
model.add(MaxPooling1D(pool_size=2))
model.add(Convolution1D(32, 3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

saved_model = "new_model.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor = "val_loss", verbose = 2, save_best_only = True, mode = 'min')

hist = model.fit(X_train, y_train, epochs = 5, batch_size = 32, validation_split=0.2, callbacks = [checkpoint], verbose = 2)

In [ ]:
# ----------------------- OBSERVED LIGHT CURVES ----------------------- #
# ----------------------- OBSERVED LIGHT CURVES ----------------------- #
# ----------------------- OBSERVED LIGHT CURVES ----------------------- #

In [4]:
# BLOK 8
# Zadefinovanie stlpcov. Nacitanie datasetu observacnych kriviek. Vyber potrebnych atributov. Vyber atributu "flux" 
# predstavujuci svietivost krivky.

columns = ['id', 'morphology', 'passband', 'params', 'data', 'origin', 'period', 'target', 'epoch', 'meta']
df = pd.read_csv(r"C:\Users\nemci\OneDrive\Počítač\Bakalárska práca\Data\observed_lc.csv", sep="|", header=None)
df.columns = columns

df = df[["target", "morphology", "data"]]
processed_data = [literal_eval(_data)["flux"] for _data in df["data"]]
morphology = df[["morphology"]].values
processed_data = np.array(processed_data, dtype=np.float32)
processed_data = np.reshape(processed_data, processed_data.shape + (1,))

In [5]:
# BLOK 9
# Vytvorenie cieloveho atributu kriviek. Hodnota 0 pre dotykove, hodnota 1 pre oddelene krivky.

target_obs = []
oc_obs, dt_obs = 0, 0
for row in df["morphology"]:
    if row == 'over-contact':
        target_obs.extend([0])
        oc_obs += 1
    elif row == 'detached':
        target_obs.extend([1])
        dt_obs += 1
target_obs = np.array(target_obs)
print('Over-contact: ' + str(oc_obs) + '\n' + 'Detached:' + str(dt_obs))

Over-contact: 47
Detached:53


In [6]:
# BLOK 10
# Nacitanie modelu. Predikovanie observacnych kriviek a vyhodnotenie modelu na observacnych datach pomocou confusion matrix a 
# vypocet presnosti, navratnosti a f1 skore

model = load_model('new_model.hdf5')
y_pred1DCNN = model.predict(processed_data)
y_pred2 = []
for i in y_pred1DCNN:
    maximum = np.argmax(i)
    y_pred2 = np.append(y_pred2, maximum)

cm = confusion_matrix(target_obs, y_pred2)
print("Confusion Matrix 1D CNN: \n" + str(cm))

prfs = classification_report(target_obs, y_pred2)
print(str(prfs))

Confusion Matrix 1D CNN: 
[[44  3]
 [ 2 51]]
              precision    recall  f1-score   support

           0       0.96      0.94      0.95        47
           1       0.94      0.96      0.95        53

    accuracy                           0.95       100
   macro avg       0.95      0.95      0.95       100
weighted avg       0.95      0.95      0.95       100

